### file 읽어오기

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader
path = r'data/medicine.txt'

# 1. loader 객체 생성
loader = TextLoader(path, encoding='utf-8')
# 2. loader를 이용해서 파일 읽어옴
docs = loader.load()

In [30]:
store = {}
config = {"configurable": {"session_id":"chat_message_1"}}

def get_session_history(session_id):
    
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]
    
    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id], # 메세지 저장소 추가.
        k=2,
        return_messages=True,
        message_key="history"
    )
    # 1. memory에서 저장된 message들을 조회
    message_list = memory.load_memory_variables({})['history']# list[Message]
    # 2. 조회한 message들을 InMemoryChatMessageHistory에 추가
    store[session_id] = InMemoryChatMessageHistory(messages=message_list)
    # 3. 2의 InMemoryChatMessageHistory를 반환.
    return store[session_id]

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.memory import ConversationBufferWindowMemory
from operator import itemgetter

# Prompt Template 생성
messages = [
            ("ai", """
            너는 유능한 약사야. context 안에 있는 내용에 한 해서
            어떤 효과가 있는지 알려주고 사용자가 입력한 증상에 맞는 약을 5개만 선택해서 알려줘.
            복용이 간단하고 부작용이 적은 약을 우선적으로 추천해서 알려줘.
            약을 복용하기 전에 반드시 알아야 할 사항은 꼭 알려줘.
            증상이 악화될 경우 대처방법에 대해서 알려줘.
            약에 따른 복용법에 대해서도 간단히 언급해줘.
            주의사항 요약해서 알려줘. 
            위의 내용을 표로 정리해서 알려줘.
            사용자가 말한 증상을 효능 부분에서 찾아서 그에 맞는 약을 알려줘.     
            {context}")"""),
            MessagesPlaceholder("history"), 
            ("human", "{question}")
    ]

prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model='gpt-4o-mini')

#output parser
parser = StrOutputParser()

def get_context(input_data:dict):
    return retriever.invoke(input_data['question'])

# Chain 구성 retriever (관련문서 조회) -> prompt_template -> model -> outputparser
runnable = {'context':RunnableLambda(get_context), 'question':itemgetter("question"), 'history':itemgetter('history')} | prompt_template | model | parser
    # RunnablePassThrough -> 이전 question 부름


chain = RunnableWithMessageHistory(
    runnable=runnable, # chain 객체(RunnableSequence)
    get_session_history=get_session_history, # session_id의 ChatMessageHistory객체를 반환하는 함수.
    input_messages_key="question", # prompt template에 입력 내용을 넣을 변수명.
    history_messages_key="history" # prompt template에 대화내역을 넣으줄 변수명
)

config = {"configurable": {"session_id":"id-1"}}
data = chain.invoke({"question":"두통인데 약 추천해줘"}  # 질문
                    , config
)

print(data)

두통에 효과적인 약물을 아래와 같이 추천합니다. 

| 약품명           | 효능                                                         | 복용법                                   | 주의사항 요약                                       |
|------------------|------------------------------------------------------------|-----------------------------------------|-----------------------------------------------------|
| 진통제 (예: 아세트아미노펜) | 두통 완화                                                   | 성인은 1회 500mg, 1일 3~4회 복용 가능 | 간 질환 환자는 사용 전 의사와 상담 필요            |
| 이부프로펜       | 두통 및 염증 완화                                          | 성인은 1회 200mg, 필요 시 4~6시간 간격으로 복용 | 위장장애나 출혈 경향이 있는 환자는 주의 필요        |
| 나프록센         | 두통과 염증에 효과적                                       | 성인은 1회 250mg, 필요 시 12시간 간격으로 복용 | 신장 질환 환자 및 위장 장애 환자는 주의 필요        |
| 아스피린         | 두통 완화 및 혈액 순환 개선                                 | 성인은 1회 300~1000mg, 필요 시 4~6시간 간격으로 복용 | 출혈 경향이 있는 환자 및 위장 장애 환자는 주의 필요 |
| 트리마졸람 (진정제) | 긴장성 두통 완화                                   

In [32]:
config = {"configurable": {"session_id":"id-1"}}
data = chain.invoke({"question":"아까 추천한 약과 소화제도 추천해줘."}  # 질문
                    , config
)

print(data)

두통과 소화불량에 효과적인 약을 아래와 같이 추천합니다. 

| 약품명               | 효능                                                     | 복용법                                         | 주의사항 요약                                       |
|----------------------|--------------------------------------------------------|------------------------------------------------|-----------------------------------------------------|
| **진통제 (예: 아세트아미노펜)** | 두통 완화                                             | 성인은 1회 500mg, 1일 3~4회 복용 가능       | 간 질환 환자는 사용 전 의사와 상담 필요            |
| **이부프로펜**       | 두통 및 염증 완화                                      | 성인은 1회 200mg, 필요 시 4~6시간 간격으로 복용 | 위장장애나 출혈 경향이 있는 환자는 주의 필요        |
| **나프록센**         | 두통과 염증에 효과적                                   | 성인은 1회 250mg, 필요 시 12시간 간격으로 복용 | 신장 질환 환자 및 위장 장애 환자는 주의 필요        |
| **베스타제정**       | 소화불량, 식욕감퇴, 소화촉진에 사용                  | 성인은 1회 2정, 1일 3회 식후 복용           | 7세 미만 영유아, 갈락토오스 불내성 환자는 복용 금지  |
| **베스타제당의정**   | 소화불량, 위부팽만감 완화에 사용              